### Generate a gallery of SDSS thumbnail pictures of fainter Perseus galaxies ###

I used [SDSS DR14 Image List Tool](http://skyserver.sdss.org/dr14/en/tools/chart/listinfo.aspx) to generate the gallery of SDSS thumbnail images.

There is a particular area in the sky that contain both dwarf galaxies in the foreground and giant galaxies in the background. My assignment is to look features that can possibly differentiate them from one another.

**Search Area:**

<img src="search_area.png">

**SQL search to filter out background galaxies from dwarf galaxies:**

```SQL
SELECT
 s.SpecObjID as name, s.ra, s.dec
FROM SpecObjAll AS s
 JOIN PhotoObjAll AS p ON s.bestObjID = p.objID
 JOIN Photoz AS ph ON ph.objID = p.objID
WHERE
 s.z BETWEEN 0.005 AND 0.030
 AND ph.z BETWEEN 0.05 AND 0.07
 AND p.ra BETWEEN (49.94-0.75) AND (49.94+0.75)
 AND p.dec BETWEEN (41.51-0.75) AND (41.51+0.75)
 AND p.type = 3 AND p.r > 16
```

In [5]:
from pandas import read_csv

In [7]:
dwrf_table = read_csv('only_dwarfs.csv')
dwrf_table

,name,ra,dec
0,1875788602521184256,50.141522,41.221872
1,1875780631061882880,50.407537,41.357786
2,1874668749676636160,49.979928,41.172643
3,1875794374957230080,49.979895,41.172622
4,1875898553683961856,50.106903,42.070372
5,1875877662963034112,49.554570,41.802521
6,1874673972356868096,49.615210,41.321313
7,1874758359874299904,49.686741,41.511440
8,1875886459056056320,49.686705,41.511441


<img src="only_dwarfs.png">

**SQL query to filter out the dwarf galaxies from the background ones:**

```SQL
SELECT
 s.SpecObjID as name, s.ra, s.dec
FROM SpecObjAll AS s
 JOIN PhotoObjAll AS p ON s.bestObjID = p.objID
 JOIN Photoz AS ph ON ph.objID = p.objID
WHERE
 s.z BETWEEN 0.04 AND 0.07
 AND ph.z BETWEEN 0.05 AND 0.07
 AND p.ra BETWEEN (49.94-0.75) AND (49.94+0.75)
 AND p.dec BETWEEN (41.51-0.75) AND (41.51+0.75)
 AND p.type = 3 AND p.r > 16
```

In [9]:
bg_table = read_csv('only_background.csv')
bg_table

,name,ra,dec
0,1875899928073496576,50.378349,41.769261
1,1874688540885936128,49.515435,41.323056
2,1875826260794435584,49.300826,41.381144
3,1875825161282807808,49.359368,41.335517


<img src="only_background.png">

Looking at the two galleries, I see a lot of similarities between them, especially in color and shape. There are two galaxies in particular from the dwarf gallery that look very close to the background ones: nr. 2 and nr. 6. In general, the galaxies in the background gallery have a more distinct core and are smaller, but it still doesn't differentiate them from the two galaxies previously mentioned. However, I see a difference: the background galaxies have a more intense red color.

What we get from this observation is that we can differentiate the background galaxies from the foreground dwarf galaxies by comparing their size and color.

### Looking for plausible Perseus galaxies ###

**SQL query that returns all spectral objects that do not have spectra:**

``` SQL
SELECT
    p.objid,p.ra,p.dec,p.u,p.g,p.r,p.i,p.z,
    ph.z AS photoZ, p.type
FROM PhotoObjAll AS p
    JOIN Photoz AS ph ON ph.objID = p.objID
WHERE
    ph.z BETWEEN 0.005 AND 0.030
    AND p.ra BETWEEN (49.94-0.75) AND (49.94+0.75)
    AND p.dec BETWEEN (41.251-0.75) AND (41.51+0.75)
    AND p.objID NOT IN(SELECT bestObjID FROM SpecObj)
```

In [26]:
table_pg = read_csv('plausible_galaxies.csv')
table_pg.head(10)

,objid,ra,dec,u,g,r,i,z,photoZ,type
0,1237661059574661461,50.645688,41.072181,24.43876,24.99780,22.87498,23.27247,21.64351,0.028931,3
1,1237661083199668321,50.646631,40.647106,12.99420,10.97638,10.08539,11.61690,10.63716,0.012487,3
2,1237661055818271052,49.212620,42.216024,23.05878,24.61830,22.22242,21.97580,22.07405,0.022111,3
3,1237661082662405095,49.386176,40.783057,23.44488,22.72720,23.09620,23.73818,22.34417,0.006613,3
4,1237661083199078605,49.214243,41.459374,17.82329,16.07929,15.30101,15.81930,14.64939,0.018372,3
5,1237661055818531546,49.855394,41.890101,24.65674,25.22565,22.65908,23.55401,22.76359,0.007608,3
6,1237661055281726387,49.575514,41.486566,18.79528,17.86434,17.51773,17.31225,17.19077,0.015904,3
7,1237661059574071741,49.285229,41.889156,24.61959,22.74701,23.91345,22.28700,22.43590,0.023748,3
8,1237661060111008912,49.768564,42.184124,19.72192,18.42307,17.96567,17.78187,17.69132,0.023422,3
9,1237670456974313590,50.066399,40.593254,22.97352,26.46167,22.38637,21.20207,20.68616,0.011076,3


**Small sample of 20 spectral objects from the automated SQL query for plausible Perseus galaxies:**

<img src="plausible_galaxies.png">